In [7]:
import tifffile
import numpy as np
import pandas as pd
import cv2
from scipy.ndimage import center_of_mass, extrema
import napari 
import matplotlib.pyplot as plt
from skimage import measure
import os

In [8]:
scalling_x = 20 #20
scalling_y = 12#25
translation_x = 200 # 422.5
translation_y = 200 # 336

#dir
main_dir = "D:/Internship/NTU/simulation/simulation_with_center/pdb/"
area_estimation_dir = "./nellie_output/simulated/low_stride/low_stride1/"

im_path = area_estimation_dir + "pdb.ome-TYX-T1p0_Y0p15_X0p15-ch0-t0_to_299-im_obj_label_reassigned.ome.tif"
path = area_estimation_dir + "pdb_event.csv"
feature_csv = area_estimation_dir +  "pdb.ome-TYX-T1p0_Y0p15_X0p15-ch0-t0_to_299-features_organelles.csv"

# load function and image

In [9]:
def get_coor (path_to_log_file,rescaleX,rescaleY):
    with open(path_to_log_file, "r", encoding='utf-8-sig') as f:
        string = f.read()
        lines = string.split("\n")
        num = 1
        event_num = []
        x_coordinates = []
        y_coordinates = []
        for line in lines:

            if len(line ) != 0:
                if (int(num) - 1)% 3  == 0: 
                    event_num.append(num)
                    num+=1
                
                else: 
                    y = line[1:-1].split(',')[1]
                    x = line[1:-1].split(',')[0]
                    y_coordinates.append((rescaleY * (-float(y)))+translation_y-scalling_y)
                    x_coordinates.append((rescaleX * float(x))+ translation_x-scalling_x)
                    #note: with center simulation Y -20 X -35
                    # 
                    num+=1
    return len(event_num) , pd.DataFrame({'y_coordinates': y_coordinates, 'x_coordinates': x_coordinates}) 

In [10]:
# Load tiff file and log position
#im_path = "D:/Internship/NTU/simulation/with_center/simulation1/multi_center_resized.ome.tif"
#im_path = main_dir + "low_stride.ome.tif"
im = tifffile.imread(im_path)
rescaling_y = im.shape[1]/400
rescaling_x = im.shape[2]/400
rescaling_y,rescaling_x

(1.0, 1.0)

In [11]:
im.shape

(300, 400, 400)

In [12]:
event_all_log = []
path_nonZero = main_dir + "nonzero_frames_final.log"
with open(path_nonZero, "r", encoding='utf-8-sig') as f:
    string = f.read()
    lines = string.split("\n")
    for line in lines:
        if len(line ) != 0:
            events = line.split(":")[1].split(",")
            frame = line.split(":")[0].split()[1]
           
            if len(events) == 1 :
                continue
            else:
                num_events = [x.split("=")[1] for x in events]
                x= int(num_events[0]) + float(num_events[1])
                num = [float(frame)-2,x,float(num_events[2])]
                event_all_log.append(num)

In [13]:
# frame , fusion, fission 
event_all_log = (np.transpose(np.array(event_all_log)))
print(f"all_fusion {np.sum(event_all_log[1])}")
print(f"all_fission {np.sum(event_all_log[2])}")


all_fusion 12.0
all_fission 10.0


In [14]:
path1_fis = main_dir + "fission_position_2.log"
path1_fus = main_dir+ "fusion_position_2.log"

num_fission,fission_coor = get_coor(path1_fis, rescaling_x, rescaling_y)
num_fusion,fusion_coor = get_coor(path1_fus, rescaling_x, rescaling_y)

print(f"num_fusion {num_fusion}")
print(f"num_fission {num_fission}")
fission_coor


num_fusion 12
num_fission 10


,y_coordinates,x_coordinates
0,135.17830,223.15840
1,134.49390,227.04860
2,244.95800,172.68748
3,249.08110,172.30324
4,156.72770,128.64220
5,160.45830,128.11510
6,181.40208,192.47900
7,177.97340,190.57540
8,160.69360,128.48670
9,156.62940,129.28620


In [15]:
# append frame in the coordinate array. 
# each frame consistingg of at least 2 coordinates, one for each fusion and fission event
fusion_frames = []
fission_frames = []

fusion_idx = 0
fission_idx = 0

for x in range(len(event_all_log[0])):
    if event_all_log[1][x] > 0 :
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[1][x]*2 )
        if x == 0:
            fusion_frames = r_frame
        else:
            fusion_frames = np.concatenate((fusion_frames,r_frame))
        fusion_idx += 1
    if event_all_log[2][x] > 0:
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[2][x]*2 )

        if x == 0:
            fission_frames = r_frame
        else:
            fission_frames = np.concatenate((fission_frames,r_frame))

        fission_idx += 1

# Assign the lists as new columns in your DataFrames
fusion_coor['frame'] = fusion_frames
fission_coor['frame'] = fission_frames

In [16]:
# need 2 loop coz frame are different
fusion_coords = []
for f in range(len(fusion_coor['frame'])):
    frame = int(fusion_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fusion_coor.iloc[f,0], fusion_coor.iloc[f,1]) # x,y coordinates or yx?
    fusion_coords.append(np.append(frame, coor))

fission_coords = []
for f in range(len(fission_coor['frame'])):
    frame = int(fission_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fission_coor.iloc[f,0], fission_coor.iloc[f,1]) # x,y coordinates or yx?
    fission_coords.append(np.append(frame, coor))


# check data


In [17]:
fission_coor

,y_coordinates,x_coordinates,frame
0,135.17830,223.15840,1.0
1,134.49390,227.04860,1.0
2,244.95800,172.68748,35.0
3,249.08110,172.30324,35.0
4,156.72770,128.64220,144.0
5,160.45830,128.11510,144.0
6,181.40208,192.47900,180.0
7,177.97340,190.57540,180.0
8,160.69360,128.48670,188.0
9,156.62940,129.28620,188.0


In [18]:
fusion_coords

[array([  6.     , 185.74517, 189.03531]),
 array([  6.     , 186.98184, 192.7074 ]),
 array([  8.    , 203.2661, 145.8333]),
 array([  8.    , 202.4806, 148.3896]),
 array([  9.    , 134.861 , 223.2359]),
 array([  9.    , 134.6771, 227.478 ]),
 array([ 15.    , 165.3222, 181.4928]),
 array([ 15.     , 163.1886 , 178.72855]),
 array([ 30.    , 199.1727, 168.6733]),
 array([ 30.     , 200.768  , 172.45567]),
 array([ 49.     , 247.0053 , 171.68413]),
 array([ 49.    , 246.7502, 171.2589]),
 array([164.    , 156.2949, 128.8944]),
 array([164.    , 160.761 , 128.2933]),
 array([191.     , 178.41332, 191.4283 ]),
 array([191.    , 176.9939, 190.7898]),
 array([196.     , 197.89665, 152.2893 ]),
 array([196.     , 194.40576, 151.1789 ]),
 array([247.    , 162.0945, 125.9883]),
 array([247.    , 157.2665, 129.0759]),
 array([276.    , 165.2708, 130.336 ]),
 array([276.    , 170.7143, 131.2755]),
 array([286.    , 172.3895, 161.5802]),
 array([286.    , 175.3092, 164.0606])]

In [19]:
fusion_coor

,y_coordinates,x_coordinates,frame
0,185.74517,189.03531,6.0
1,186.98184,192.70740,6.0
2,203.26610,145.83330,8.0
3,202.48060,148.38960,8.0
4,134.86100,223.23590,9.0
5,134.67710,227.47800,9.0
6,165.32220,181.49280,15.0
7,163.18860,178.72855,15.0
8,199.17270,168.67330,30.0
9,200.76800,172.45567,30.0


# Load fission fusion frome my script

In [20]:
df = pd.read_csv(path)

mask_fusion = df['isFusion'].astype(int) == 1
fission_event = df[~mask_fusion]
fusion_event = df[mask_fusion]

In [21]:
fission_event

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label
0,0.0,0.0,0.0,0.0,117.755725,218.374046,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.0,134.930693,254.930693,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,135.809091,113.609091,0.0,0.0,144.0,2.0
4,0.0,0.0,0.0,0.0,137.161905,113.800000,0.0,0.0,188.0,2.0
5,0.0,0.0,0.0,0.0,137.008850,113.035398,0.0,0.0,235.0,2.0
6,0.0,0.0,0.0,0.0,159.040816,114.734694,0.0,0.0,235.0,2.0
8,0.0,0.0,0.0,0.0,183.276503,146.874317,0.0,0.0,284.0,2.0
9,0.0,0.0,0.0,0.0,179.205420,152.767483,0.0,0.0,287.0,2.0
10,0.0,0.0,0.0,0.0,170.835821,190.746269,0.0,0.0,5.0,3.0
11,0.0,0.0,0.0,0.0,189.659955,167.599553,0.0,0.0,5.0,3.0


In [22]:
fusion_event

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label
2,0.00,0.000,0.000000,0.0,129.358242,244.024176,0.00,1.0,8.0,1.0
7,0.00,0.000,0.000000,0.0,144.046053,113.868421,0.00,1.0,247.0,2.0
12,0.00,0.000,0.000000,0.0,185.307958,172.852941,0.00,1.0,6.0,3.0
13,0.45,0.405,43.054238,5.0,189.000000,180.000000,2.16,1.0,171.0,3.0
16,0.00,0.000,0.000000,0.0,203.238064,160.275343,0.00,1.0,196.0,3.0
24,0.00,0.000,0.000000,0.0,188.562500,180.000000,0.00,1.0,163.0,5.0


In [23]:
df_fission = pd.DataFrame({
    'frame': fission_event['Frame'],
    'x_coordinates': fission_event['x_corr'],
    'y_coordinates': fission_event['y_corr'],
})

df_fusion = pd.DataFrame({
    'frame': fusion_event['Frame'],
    'x_coordinates': fusion_event['x_corr'],
    'y_coordinates': fusion_event['y_corr'],
})

In [24]:
df[df['Frame'] == 9]

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label


In [25]:
fis_arr = df_fission.to_numpy()
fis_arr = fis_arr[:,[0,2,1]] # frame, y_coordinates, x_coordinates
fus_arr= df_fusion.to_numpy()# frame, y_coordinates, x_coordinates
fus_arr = fus_arr[:,[0,2,1]] # frame, y_coordinates, x_coordinates
list_fis = []
list_fis = [np.array(fis_list) for fis_list in fis_arr]
list_fus = []
list_fus = [np.array(fus_list) for fus_list in fus_arr]

In [26]:
list_fus

[array([  8.        , 129.35824176, 244.02417582]),
 array([247.        , 144.04605263, 113.86842105]),
 array([  6.        , 185.30795848, 172.85294118]),
 array([171., 189., 180.]),
 array([196.        , 203.23806409, 160.27534336]),
 array([163.    , 188.5625, 180.    ])]

In [27]:
list_fis

[array([  1.        , 117.75572519, 218.3740458 ]),
 array([  1.        , 134.93069307, 254.93069307]),
 array([144.        , 135.80909091, 113.60909091]),
 array([188.        , 137.16190476, 113.8       ]),
 array([235.        , 137.00884956, 113.03539823]),
 array([235.        , 159.04081633, 114.73469388]),
 array([284.        , 183.27650273, 146.87431694]),
 array([287.        , 179.20541958, 152.76748252]),
 array([  5.        , 170.8358209 , 190.74626866]),
 array([  5.        , 189.65995526, 167.59955257]),
 array([194.        , 204.80904908, 161.13190184]),
 array([194.        , 183.99115044, 150.91150442]),
 array([286.        , 179.25440806, 152.69437448]),
 array([296.        , 177.95689655, 179.95689655]),
 array([180.        , 165.75088968, 166.00711744]),
 array([279.        , 167.20394737, 168.18421053]),
 array([  0. , 189. , 179.5]),
 array([162. , 189. , 178.5]),
 array([162. , 189. , 181.5]),
 array([  0. , 189. , 182.5]),
 array([ 73. , 189. , 182.5]),
 array([ 82. 

# visualize tiff

In [28]:
middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    middle = np.vstack((middle, new_row))

tiff = "D:/Internship/NTU/simulation/simulation_with_center/pdb/pdb.ome.tif"
im_tif = tifffile.imread(tiff)
n_frames = im_tif.shape[0]

In [29]:
square_size = 10  # pixels
image_resolution = [400, 400] 
square_vertices = [
    [
        [t, 0, 0],  # Top-left
        [t, 0, image_resolution[0]],  # Top-right
        [t, image_resolution[1], image_resolution[0]],  # Bottom-right
        [t, image_resolution[1], 0]  # Bottom-left
    ]
    for t in range(n_frames)
]

In [30]:
viewer = napari.Viewer()
viewer.add_labels(im_tif, name ="labeled image")
viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(fusion_coords, face_color='yellow', size =2, name = "fusion_node")
viewer.add_points(fission_coords, face_color='yellow', size =2, name = "fission_node")
viewer.add_shapes(
    square_vertices,
    shape_type="rectangle",
    edge_color="red",
    face_color="transparent",
    edge_width=2,
    name="Frame Boundary Square"
)

<Shapes layer 'Frame Boundary Square' at 0x26b169e3970>

# run visualization

In [131]:
middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    middle = np.vstack((middle, new_row))

In [132]:
frames = im.shape[0]
centroids = []
labels= []

nellie_df_2d = pd.read_csv(feature_csv)

for frame in range(frames):
        labeled_im_frame = im[frame]
        unique_label = np.unique(labeled_im_frame)
        unique_label = unique_label[unique_label != 0]
        num_features = len(unique_label) 

        # !!!!!cannot find a label since it always start over if we use range(len(unique_label))
        for label in unique_label:
            all_extrema = center_of_mass(labeled_im_frame == label)
            centroid = all_extrema#[3]
            centroid = np.array(list(centroid))
            centroids.append(np.append(frame, centroid))
            
            nellie_df_2d_label = nellie_df_2d[nellie_df_2d['label'] == label]
            reassigned_label = nellie_df_2d_label['reassigned_label_raw']
            labels.append(int(list(reassigned_label)[0]))
        #add number of assigned label
            #if label == contour_label:
             #       contours = get_label_contour(labeled_im_frame, label)
              #      for contour in contours:
               #         contour_points.append(np.column_stack((np.full(len(contour), frame), contour)))

    

In [133]:
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(fusion_coords, face_color='yellow', size =2, name = "fusion_node")
viewer.add_points(fission_coords, face_color='yellow', size =2, name = "fission_node")

viewer.add_points(list_fis, face_color='green', size =2, name = "fission_area")
viewer.add_points(list_fus, face_color='green', size =2, name = "fusion_area")

viewer.add_points(centroids, text = labels, size =0.5, name = "label number")
napari.run()

#TODO is it the correct position of this iamghe

# Back up

In [ ]:
'''
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =5, name = "label number")
viewer.add_points(test_coord, face_color='yellow', size =1 , text = label, name = "label number")
napari.run()
'''


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

# Create a 2D binary image
img = np.array([[0, 2, 2, 0],
                [0, 2, 2, 0],
                [0, 2, 2, 0],
                [0, 0, 0, 0]])

# Calculate center of mass
com = ndimage.center_of_mass(img)
print(com)
# Plot the image and center of mass
plt.imshow(img, cmap='gray')
plt.scatter(com[1], com[0], c='red', marker='x', s=100)  # Swap row, column for plotting
plt.title('Center of Mass')
plt.show()